In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import random
import re
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob,Word, Blobber
nltk.download('stopwords')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import f1_score, accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [26]:
# Read test data
df = pd.read_csv('saltyhacker.csv')
df.head(10)


,Unnamed: 0,User,Text
0,0,thu2111,Did those people criticise other epidemiologis...
1,1,noisy_boy,I like Aliexpress because most of the times I ...
2,2,lopis,Doesn&#x27;t really matter. The progression of...
3,3,aexol,I have added docs generator for GraphQL Editor...
4,4,frockington1,I haven;t seen any non sensationalized evidenc...
5,5,alpaca128,&gt; Amazon is the only company that reliably ...
6,6,alkonaut,Isn&#x27;t this just a normal water heater?<p>...
7,7,giantg2,"Yes, that&#x27;s a textbook scenario that make..."
8,8,9wzYQbTYsAIc,There’s something about JavaScript and the len...
9,9,Jonnax,What would they be researching?<p>Especially w...


In [0]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
def sentiment_value(paragraph):
    analyser = SentimentIntensityAnalyzer()
    result = analyser.polarity_scores(paragraph)
    score = result['compound']
    return round(score,1)

In [33]:
sample = df['Text'][1243]
print(sample)
print('Sentiment: ')
print(sentiment_value(sample))

My point is about rates... COVID is less lethal than the Flu even if it&#x27;s temporarily causing more damage because it&#x27;s new.<p>You said nothing about death rates? I did... the rates are very important when it comes to decisions about whether or not we should lose our collective shit over it or not.<p>Something that&#x27;s more communicable than others but less lethal than the flu? We can&#x27;t contain it and shutting the country down is going to destroy more lives and cause more damage than COVID.<p>My original post was that COVID is less lethal. And it is turning out, that with more information coming out each day, this is true.<p>My definition of Less lethal = deaths &#x2F; infection count.<p>I&#x27;d also add that it&#x27;s hard to take current COVID death counts serious when hospitals across the country get paid more for COVID cases.<p>It&#x27;s hard to not see hospitals that are hurting for money turn down 30% more money for the same cases.
Sentiment: 
-0.9


In [0]:
def print_accuracy(dhn, pred_column):
    "Print accuracy after making predictions"
    acc = accuracy_score(['Text'], df[pred_column])*100
    return acc

In [0]:
#textblob
def textblob_score(sentence):
    return TextBlob(sentence).sentiment.polarity

In [0]:
#convert to string
df['Text'] = df['Text'].astype(str)

In [10]:
# Convert textblob sentiment score
df['textblob_score'] = df['Text'].apply(textblob_score)
# Convert float score to category based on binning
df['textblob_pred'] = pd.cut(df['textblob_score'], bins=5, labels=[1, 2, 3, 4, 5])
df = df.drop('textblob_score', axis=1)
df.head()

,Unnamed: 0,User,Text,textblob_pred
0,0,thu2111,Did those people criticise other epidemiologis...,3
1,1,noisy_boy,I like Aliexpress because most of the times I ...,3
2,2,lopis,Doesn&#x27;t really matter. The progression of...,3
3,3,aexol,I have added docs generator for GraphQL Editor...,4
4,4,frockington1,I haven;t seen any non sensationalized evidenc...,4


In [0]:
# # #Get model accuracy
# acc = print_accuracy(df, 'textblob_pred')
# print("Accuracy: {}".format(acc[0], acc[1]))

In [11]:
import nltk
nltk.download('vader_lexicon')
vader = SentimentIntensityAnalyzer()
def score_vader(sentence, vader):
    return vader.polarity_scores(sentence)['compound']

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [12]:
# Calculate Vader sentiment score
df['vader_score'] = df['Text'].apply(lambda x: score_vader(x, vader))
# Convert float score to category based on binning
df['vader_pred'] = pd.cut(df['vader_score'], bins=5, labels=[1, 2, 3, 4, 5])
df = df.drop('vader_score', axis=1)
df.head()

,Unnamed: 0,User,Text,textblob_pred,vader_pred
0,0,thu2111,Did those people criticise other epidemiologis...,3,1
1,1,noisy_boy,I like Aliexpress because most of the times I ...,3,3
2,2,lopis,Doesn&#x27;t really matter. The progression of...,3,3
3,3,aexol,I have added docs generator for GraphQL Editor...,4,3
4,4,frockington1,I haven;t seen any non sensationalized evidenc...,4,3


In [0]:
# # # Get model accuracy 
# acc = print_accuracy(df, 'vader_pred')
# print("Accuracy: {}".format(acc[0], acc[1]))



# # accuracy: (tp + tn) / (p + n)
# accuracy = accuracy_score(df, 'vader_pred')
# print('Accuracy: %f' % accuracy)

# # f1: 2 tp / (2 tp + fp + fn)
# f1 = f1_score(df, 'vader_pred')
# print('F1 score: %f' % f1)